In [18]:
!mv dalle2_pytorch.py /home/user/conda/envs/iivii/lib/python3.10/site-packages/dalle2_pytorch/dalle2_pytorch.py

mv: cannot stat 'dalle2_pytorch.py': No such file or directory


In [19]:
from dalle2_laion import ModelLoadConfig, DalleModelManager
from dalle2_laion.scripts import InferenceScript

class ExampleInference(InferenceScript):
    def run(self, text: str):
        """
        Takes a string and returns a single image.
        """
        text = [text]
        image_embedding_map = self._sample_prior(text)
        image_embedding = image_embedding_map[0][0].unsqueeze(0)
        image_map = self._sample_decoder(text=text, image_embed=image_embedding)
        return image_map[0][0]

# model_config = ModelLoadConfig.from_json_path("dalle2_laion.json")
# model_manager = DalleModelManager(model_config)
# inference = ExampleInference(model_manager)
# image = inference.run("Hello World")

In [20]:
model_config = ModelLoadConfig.from_json_path("dalle2_laion.json")

In [8]:
model_manager = DalleModelManager(model_config)

FIX: Switch to this version with `pip install DALLE2-pytorch==1.1.0`. If different models suggest different versions, you may just need to choose one.


In [9]:
inference = ExampleInference(model_manager)

In [10]:
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision.transforms import ToTensor, ToPILImage
import numpy as np

import os
from torchvision.transforms import Compose, ToTensor, Pad, Resize, ToPILImage, InterpolationMode

class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, im_dir, caption_dir):
        self.im_dir = im_dir
        self.caption_dir = caption_dir
        self.im_fnames = sorted(os.listdir(im_dir))
        self.caption_fnames = sorted(os.listdir(caption_dir))
        self.to_tensor_transform = ToTensor()
        self.pad_transform = lambda im, pad_right, pad_bottom: Pad(padding=(0, 0, pad_right, pad_bottom))(im)
        self.resize_transform = Resize((256, 256), interpolation=InterpolationMode.BILINEAR)

    def __getitem__(self, idx):
        im = Image.open(os.path.join(self.im_dir, self.im_fnames[idx]))
        w, h = im.size
        im = self.to_tensor_transform(im)
        new_size = max(w, h)
        im = self.pad_transform(im, new_size - w, new_size - h)
        im = self.resize_transform(im)

        with open(os.path.join(self.caption_dir, self.caption_fnames[idx]), 'r') as captions_f:
            captions = captions_f.readlines()

        return self.im_fnames[idx], im, captions

    def __len__(self):
        return len(self.im_fnames)

In [11]:
from functools import partial, wraps
from tqdm.auto import tqdm
from contextlib import contextmanager
from dalle2_pytorch.vqgan_vae import NullVQGanVAE, VQGanVAE

def exists(val):
    return val is not None

def first(arr, d = None):
    if len(arr) == 0:
        return d
    return arr[0]

def maybe(fn):
    @wraps(fn)
    def inner(x, *args, **kwargs):
        if not exists(x):
            return x
        return fn(x, *args, **kwargs)
    return inner

def default(val, d):
    if exists(val):
        return val
    return d() if callable(d) else d

def cast_tuple(val, length = None, validate = True):
    if isinstance(val, list):
        val = tuple(val)

    out = val if isinstance(val, tuple) else ((val,) * default(length, 1))

    if exists(length) and validate:
        assert len(out) == length

    return out

@contextmanager
def null_context(*args, **kwargs):
    yield

def eval_decorator(fn):
    def inner(model, *args, **kwargs):
        was_training = model.training
        model.eval()
        out = fn(model, *args, **kwargs)
        model.train(was_training)
        return out
    return inner

def resize_image_to(
    image,
    target_image_size,
    clamp_range = None,
    nearest = False,
    **kwargs
):
    orig_image_size = image.shape[-1]

    if orig_image_size == target_image_size:
        return image

    if not nearest:
        scale_factors = target_image_size / orig_image_size
        out = resize(image, scale_factors = scale_factors, **kwargs)
    else:
        out = F.interpolate(image, target_image_size, mode = 'nearest')

    if exists(clamp_range):
        out = out.clamp(*clamp_range)

    return out

In [12]:
inference.model_manager.decoder_info.model.cuda()

Decoder(
  (unets): ModuleList(
    (0): Unet(
      (init_conv): CrossEmbedLayer(
        (convs): ModuleList(
          (0): Conv2d(3, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): Conv2d(3, 80, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
          (2): Conv2d(3, 80, kernel_size=(15, 15), stride=(1, 1), padding=(7, 7))
        )
      )
      (to_time_hiddens): Sequential(
        (0): SinusoidalPosEmb()
        (1): Linear(in_features=320, out_features=1280, bias=True)
        (2): GELU(approximate='none')
      )
      (to_time_tokens): Sequential(
        (0): Linear(in_features=1280, out_features=1024, bias=True)
        (1): Rearrange('b (r d) -> b r d', r=2)
      )
      (to_time_cond): Sequential(
        (0): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (image_to_tokens): Sequential(
        (0): Linear(in_features=768, out_features=2048, bias=True)
        (1): Rearrange('b (n d) -> b n d', n=4)
      )
      (to_image

In [21]:
def my_sample(
    self,
    lowres_cond_img=None,
    image = None,
    image_embed = None,
    text = None,
    text_encodings = None,
    batch_size = 1,
    cond_scale = 1.,
    start_at_unet_number = 1,
    stop_at_unet_number = None,
    distributed = False,
    inpaint_image = None,
    inpaint_mask = None,
    inpaint_resample_times = 5,
):
    assert self.unconditional or exists(image_embed), 'image embed must be present on sampling from decoder unless if trained unconditionally'

    if not self.unconditional:
        batch_size = image_embed.shape[0]

    if exists(text) and not exists(text_encodings) and not self.unconditional:
        assert exists(self.clip)
        _, text_encodings = self.clip.embed_text(text)

    assert not (self.condition_on_text_encodings and not exists(text_encodings)), 'text or text encodings must be passed into decoder if specified'
    assert not (not self.condition_on_text_encodings and exists(text_encodings)), 'decoder specified not to be conditioned on text, yet it is presented'

    assert not (exists(inpaint_image) ^ exists(inpaint_mask)), 'inpaint_image and inpaint_mask (boolean mask of [batch, height, width]) must be both given for inpainting'

    img = None
    if start_at_unet_number > 1:
        # Then we are not generating the first image and one must have been passed in
        assert exists(image), 'image must be passed in if starting at unet number > 1'
        assert image.shape[0] == batch_size, 'image must have batch size of {} if starting at unet number > 1'.format(batch_size)
        prev_unet_output_size = self.image_sizes[start_at_unet_number - 2]
        img = resize_image_to(image, prev_unet_output_size, nearest = True)
    is_cuda = next(self.parameters()).is_cuda
    num_unets = self.num_unets
    cond_scale = cast_tuple(cond_scale, num_unets)

    for unet_number, unet, vae, channel, image_size, predict_x_start, learned_variance, noise_scheduler, lowres_cond, sample_timesteps, unet_cond_scale in tqdm(zip(range(1, num_unets + 1), self.unets, self.vaes, self.sample_channels, self.image_sizes, self.predict_x_start, self.learned_variance, self.noise_schedulers, self.lowres_conds, self.sample_timesteps, cond_scale)):
        if unet_number < start_at_unet_number:
            continue  # It's the easiest way to do it

        context = self.one_unet_in_gpu(unet = unet) if is_cuda else null_context()

        with context:
            # prepare low resolution conditioning for upsamplers

            #lowres_cond_img = lowres_noise_level = None
            lowres_noise_level = None
            shape = (batch_size, channel, image_size, image_size)

#             if unet.lowres_cond:
#                 lowres_cond_img = resize_image_to(img, target_image_size = image_size, clamp_range = self.input_image_range, nearest = True)

#                 if lowres_cond.use_noise:
#                     lowres_noise_level = torch.full((batch_size,), int(self.lowres_noise_sample_level * 1000), dtype = torch.long, device = self.device)
#                     lowres_cond_img, _ = lowres_cond.noise_image(lowres_cond_img, lowres_noise_level)

            # latent diffusion

            is_latent_diffusion = isinstance(vae, VQGanVAE)
            image_size = vae.get_encoded_fmap_size(image_size)
            shape = (batch_size, vae.encoded_dim, image_size, image_size)

            lowres_cond_img = maybe(vae.encode)(lowres_cond_img)

            # denoising loop for image

            img = self.p_sample_loop(
                unet,
                shape,
                image_embed = image_embed,
                text_encodings = text_encodings,
                cond_scale = unet_cond_scale,
                predict_x_start = predict_x_start,
                learned_variance = learned_variance,
                clip_denoised = not is_latent_diffusion,
                lowres_cond_img = lowres_cond_img,
                lowres_noise_level = lowres_noise_level,
                is_latent_diffusion = is_latent_diffusion,
                noise_scheduler = noise_scheduler,
                timesteps = sample_timesteps,
                inpaint_image = inpaint_image,
                inpaint_mask = inpaint_mask,
                inpaint_resample_times = inpaint_resample_times
            )

            img = vae.decode(img)

        if exists(stop_at_unet_number) and stop_at_unet_number == unet_number:
            break

    return img


In [22]:
inference.model_manager.decoder_info.model.sample_timesteps = (None, 1000)

In [23]:
def psnr(pred, gt):
    # pred \in [0, 1]
    pred_int = pred * 255
    gt = gt * 255
    pred_int = torch.round(pred_int)
    gt = torch.round(gt)
    return 20 * torch.log10(255 / torch.sqrt(F.mse_loss(pred_int, gt)))

## Start

In [25]:
dataset = CocoDataset(
    im_dir="images/",
    caption_dir="captions/"
)

psnrs = []

for source_image_i in range(len(dataset)):
    (source_image_name, source_image, source_captions) = dataset[source_image_i]
    max_psnr = 0
    for text_str_i, text_str in enumerate(source_captions):
        text = [text_str]

        with torch.no_grad():
            image_embedding_map = inference._sample_prior(text)
            image_embedding = image_embedding_map[0][0].unsqueeze(0)
            source_image_small_64 = F.interpolate(source_image.unsqueeze(0), size=(64, 64))
        source_image_small_256 = F.interpolate(source_image_small_64, size=(256, 256))
        # normalize
#         source_image_small_64 = source_image_small_64 * 2 - 1
#         source_image_small_256 = source_image_small_256 * 2 - 1

        #source_image_small_64.shape, source_image_small_256.shape

        inference.model_manager.decoder_info.model.cuda()

        source_image_small_256 = source_image_small_256.cuda()
        with torch.no_grad():
            text_encodings = inference._encode_text(text).cuda()
        image_embed = image_embedding.cuda()
        source_image_small_64 = source_image_small_64.cuda()
        source_image = source_image.cuda()

        text_encodings.requires_grad = True
        image_embed.requires_grad = True

        source_image_small_64.requires_grad = True
        source_image_small_256.requires_grad = True
        source_image.requires_grad = False

        optimizer = torch.optim.AdamW([
            source_image_small_64, source_image_small_256, text_encodings, image_embed
        ], lr = 1.0)
        n_iters = 2
        scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.5, total_iters=n_iters)
        
        for i in range(n_iters - 1):
            inference.model_manager.decoder_info.model.train()
            optimizer.zero_grad()
            res = my_sample(
                self=inference.model_manager.decoder_info.model,
                lowres_cond_img=source_image_small_256,
                image_embed = image_embed,
                text_encodings = text_encodings,
                image=source_image_small_64,
                start_at_unet_number = 2,
            )
            res.retain_grad()
            decoded_coco_image = res.squeeze(0)
            loss = F.mse_loss(decoded_coco_image, source_image)
            loss.backward()
            optimizer.step()
            scheduler.step()
            inference.model_manager.decoder_info.model.eval()
            with torch.no_grad():
                max_psnr = max(psnr(decoded_coco_image, source_image).item(), max_psnr)
    print(f"image = {source_image_name}")
    print(max_psnr)
    print("\n---------------------------\n")
    psnrs.append(max_psnr)

sampling loop time step:   0%|          | 0/64 [00:00<?, ?it/s]

0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/64 [00:00<?, ?it/s]

0it [00:00, ?it/s]

...

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/64 [00:00<?, ?it/s]

0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

image = tie.jpg
29.001123424938838

---------------------------



In [26]:
import numpy as np

print(np.mean(psnrs))

30.051281808587067
